# 1. Загрузка датасета

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import corus
import wget
import os
import ssl

url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
filename = "lenta-ru-news.csv.gz"

if not os.path.exists(filename):
    wget.download(url)

records = corus.load_lenta(filename)

df = pd.DataFrame(records)

df.columns = ["url", "title", "text", "topic", "tags", "extra"]

df = df[["title", "text", "topic"]]

df.head()

,title,text,topic
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир


In [3]:
df.shape

(739351, 3)

In [16]:
df["topic"].value_counts(normalize=True) * 100

topic
Россия               21.710798
Мир                  18.486483
Экономика            10.757813
Спорт                 8.713182
Культура              7.277058
Бывший СССР           7.222821
Наука и техника       7.186844
Интернет и СМИ        6.042462
Из жизни              3.734491
Дом                   2.939605
Силовые структуры     2.650433
Ценности              1.050381
Бизнес                1.000743
Путешествия           0.866706
69-я параллель        0.171502
Крым                  0.090079
Культпросвет          0.045986
                      0.027457
Легпром               0.015419
Библиотека            0.008791
Оружие                0.000406
ЧМ-2014               0.000271
МедНовости            0.000135
Сочи                  0.000135
Name: proportion, dtype: float64

# 2. Подготовка данных

## 2.1. Создание стратифицированной выборки

In [4]:
# Создаем стратифицированную выборку из 100 000 строк
from sklearn.model_selection import train_test_split

sample_size = 100000
random_state = 42

# Находим топики с количеством элементов >= 2
topic_counts = df["topic"].value_counts()
valid_topics = topic_counts[topic_counts >= 2].index.tolist()

df_filtered = df[df["topic"].isin(valid_topics)]

# Получаем стратифицированную выборку
df_sample, _ = train_test_split(
    df_filtered,
    train_size=sample_size,
    stratify=df_filtered["topic"],
    random_state=random_state,
)

print(f"Размер выборки: {df_sample.shape[0]} строк")

topic_distribution = df_sample["topic"].value_counts(normalize=True) * 100
print("\nРаспределение топиков в выборке (%):")
print(topic_distribution)

original_distribution = df_filtered["topic"].value_counts(normalize=True) * 100
print("\nИсходное распределение топиков (%):")
print(original_distribution)

print("\nРазница между распределениями (процентные пункты):")
diff = topic_distribution - original_distribution
print(diff.abs().sort_values(ascending=False))

Размер выборки: 100000 строк

Распределение топиков в выборке (%):
topic
Россия               21.711
Мир                  18.487
Экономика            10.758
Спорт                 8.713
Культура              7.277
Бывший СССР           7.223
Наука и техника       7.187
Интернет и СМИ        6.042
Из жизни              3.735
Дом                   2.940
Силовые структуры     2.650
Ценности              1.050
Бизнес                1.001
Путешествия           0.867
69-я параллель        0.172
Крым                  0.090
Культпросвет          0.046
                      0.027
Легпром               0.015
Библиотека            0.009
Name: proportion, dtype: float64

Исходное распределение топиков (%):
topic
Россия               21.710856
Мир                  18.486533
Экономика            10.757842
Спорт                 8.713206
Культура              7.277078
Бывший СССР           7.222841
Наука и техника       7.186863
Интернет и СМИ        6.042478
Из жизни              3.734502
Дом         

## 2.2. Нормализация текста

In [5]:
import re


def normalize_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Замена множественных пробелов одиночными
    text = re.sub(r"\s+", " ", text)

    # Удаление пробелов в начале и конце
    text = text.strip()

    return text


df_sample["title_norm"] = df_sample["title"].apply(normalize_text)
df_sample["text_norm"] = df_sample["text"].apply(normalize_text)

## 2.3. Удаление стоп-слов

In [6]:
from nltk.corpus import stopwords
import nltk

try:
    russian_stopwords = stopwords.words("russian")
except:
    nltk.download("stopwords")
    russian_stopwords = stopwords.words("russian")


def clean_text(text):
    # Удаление стоп-слов
    words = text.split()
    words = [word for word in words if word not in russian_stopwords]

    return " ".join(words)


df_sample["title_clean"] = df_sample["title_norm"].apply(clean_text)
df_sample["text_clean"] = df_sample["text_norm"].apply(clean_text)

In [34]:
print(df_sample.iloc[0].text)

Государственный департамент США заявил, что заявления Арафата, призывающего палестинцев прекратить теракты против граждан Израиля, не достаточно для того, чтобы прекратить насилие, сообщает Ha'aretz. "Заявление Арафата, осуждающее взрывы террористов-самоубийц является шагом в правильном направлении, - говорится в заявлении департамента. - Но очевидно, что следует сделать гораздо больше для того, чтобы прекратить насилие и террор". Представитель Белого Дома Ари Флейшер (Ari Fleisher) также заявил, что президент Буш ждет от Арафата не риторики, а реальных действий. Руководство Израиля, со своей стороны, заявило, что осуждение терактов является со стороны Арафата лишь лицемерием, так как на деле он и пальцем не двинет для того, чтобы действительно остановить террор.


In [35]:
print(df_sample.iloc[0].text_norm)

государственный департамент сша заявил, что заявления арафата, призывающего палестинцев прекратить теракты против граждан израиля, не достаточно для того, чтобы прекратить насилие, сообщает ha'aretz. "заявление арафата, осуждающее взрывы террористов-самоубийц является шагом в правильном направлении, - говорится в заявлении департамента. - но очевидно, что следует сделать гораздо больше для того, чтобы прекратить насилие и террор". представитель белого дома ари флейшер (ari fleisher) также заявил, что президент буш ждет от арафата не риторики, а реальных действий. руководство израиля, со своей стороны, заявило, что осуждение терактов является со стороны арафата лишь лицемерием, так как на деле он и пальцем не двинет для того, чтобы действительно остановить террор.


In [45]:
print(df_sample.iloc[0].text_clean)

государственный департамент сша заявил, заявления арафата, призывающего палестинцев прекратить теракты против граждан израиля, достаточно того, прекратить насилие, сообщает ha'aretz. "заявление арафата, осуждающее взрывы террористов-самоубийц является шагом правильном направлении, - говорится заявлении департамента. - очевидно, следует сделать гораздо того, прекратить насилие террор". представитель белого дома ари флейшер (ari fleisher) также заявил, президент буш ждет арафата риторики, реальных действий. руководство израиля, своей стороны, заявило, осуждение терактов является стороны арафата лишь лицемерием, деле пальцем двинет того, действительно остановить террор.


- в russian_stopwords есть некоторые слова, устранение которых может исказить или поменять смысл текста, например "не", "ни", "нет"

In [7]:
russian_stopwords.remove("нет")
russian_stopwords.remove("не")
russian_stopwords.remove("ни")
russian_stopwords.remove("без")
russian_stopwords.remove("только")
russian_stopwords.remove("даже")
russian_stopwords.remove("совсем")
russian_stopwords.remove("почти")
russian_stopwords.remove("всегда")
russian_stopwords.remove("никогда")
russian_stopwords.remove("ничего")
russian_stopwords.remove("нельзя")
russian_stopwords.remove("более")
russian_stopwords.remove("больше")
russian_stopwords.remove("лучше")

In [8]:
df_sample["title_clean"] = df_sample["title_norm"].apply(clean_text)
df_sample["text_clean"] = df_sample["text_norm"].apply(clean_text)

## 2.4. Лемматизация

Для русского языка лемматизация обычно предпочтительнее стемминга из-за богатой морфологии.

In [9]:
from pymystem3 import Mystem

mystem = Mystem()


def lemmatize_text(text):
    lemmas = mystem.lemmatize(text)
    return "".join(lemmas).strip()


df_sample["title_lemma"] = df_sample["title_clean"].apply(lemmatize_text)
df_sample["text_lemma"] = df_sample["text_clean"].apply(lemmatize_text)

In [51]:
print(df_sample.iloc[0].text_lemma)

государственный департамент сша заявлять, заявление арафат, призывать палестинец прекращать теракт против гражданин израиль, не достаточно то, прекращать насилие, сообщать ha'aretz. "заявление арафат, осуждать взрыв террорист-самоубийца являться шаг правильный направление, - говориться заявление департамент. - очевидно, следовать сделать гораздо много то, прекращать насилие террор". представитель белый дом ари флейшер (ari fleisher) также заявлять, президент буш ждать арафат не риторика, реальный действие. руководство израиль, свой сторона, заявлять, осуждение теракт являться сторона арафат лишь лицемерие, дело палец не двинуть то, действительно останавливать террор.


## 2.5. Преобразование таргета

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_sample["topic_encoded"] = label_encoder.fit_transform(df_sample["topic"])

label_dict = dict(
    zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_)
)
print("Словарь меток:", label_dict)

Словарь меток: {0: '', 1: '69-я параллель', 2: 'Библиотека', 3: 'Бизнес', 4: 'Бывший СССР', 5: 'Дом', 6: 'Из жизни', 7: 'Интернет и СМИ', 8: 'Крым', 9: 'Культпросвет ', 10: 'Культура', 11: 'Легпром', 12: 'Мир', 13: 'Наука и техника', 14: 'Путешествия', 15: 'Россия', 16: 'Силовые структуры', 17: 'Спорт', 18: 'Ценности', 19: 'Экономика'}


## 2.6. Train-test split


In [11]:
X = df_sample[["title_lemma", "text_lemma"]]
y = df_sample["topic_encoded"]

# train (60%) и temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=random_state, stratify=y
)

# validation (50%) и test (50%) от temp
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=random_state, stratify=y_temp
)

print(
    f"Размер обучающей выборки: {X_train.shape[0]} строк ({X_train.shape[0]/len(X)*100:.1f}%)"
)
print(
    f"Размер валидационной выборки: {X_val.shape[0]} строк ({X_val.shape[0]/len(X)*100:.1f}%)"
)
print(
    f"Размер тестовой выборки: {X_test.shape[0]} строк ({X_test.shape[0]/len(X)*100:.1f}%)"
)

print("\nРаспределение классов в обучающей выборке:")
print(y_train.value_counts(normalize=True).sort_index() * 100)

print("\nРаспределение классов в валидационной выборке:")
print(y_val.value_counts(normalize=True).sort_index() * 100)

print("\nРаспределение классов в тестовой выборке:")
print(y_test.value_counts(normalize=True).sort_index() * 100)

Размер обучающей выборки: 60000 строк (60.0%)
Размер валидационной выборки: 20000 строк (20.0%)
Размер тестовой выборки: 20000 строк (20.0%)

Распределение классов в обучающей выборке:
topic_encoded
0      0.026667
1      0.171667
2      0.008333
3      1.001667
4      7.223333
5      2.940000
6      3.735000
7      6.041667
8      0.090000
9      0.046667
10     7.276667
11     0.015000
12    18.486667
13     7.186667
14     0.866667
15    21.711667
16     2.650000
17     8.713333
18     1.050000
19    10.758333
Name: proportion, dtype: float64

Распределение классов в валидационной выборке:
topic_encoded
0      0.030
1      0.175
2      0.010
3      1.000
4      7.220
5      2.940
6      3.735
7      6.040
8      0.090
9      0.045
10     7.280
11     0.015
12    18.490
13     7.185
14     0.870
15    21.710
16     2.650
17     8.710
18     1.050
19    10.755
Name: proportion, dtype: float64

Распределение классов в тестовой выборке:
topic_encoded
0      0.025
1      0.170
2      0.0

Пайплайн обработки включает нормализацию (приведение к нижнему регистру), очистку от стоп-слов с сохранением ключевых смысловых частиц (отрицаний), лемматизацию как оптимальный метод для русского языка и стратифицированное разделение данных для сохранения пропорций классов.

## 2.7 Save data

In [13]:
X_train.to_csv('../data/X_train.csv', index=False)
X_val.to_csv('../data/X_val.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)

y_train.to_csv('../data/y_train.csv', index=False)
y_val.to_csv('../data/y_val.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

# 3. Dummy baseline

In [68]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

dummy_clf = DummyClassifier(strategy="most_frequent", random_state=random_state)
dummy_clf.fit(X_train, y_train)

y_val_pred = dummy_clf.predict(X_val)

print(classification_report(y_val, y_val_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00       200
           4       0.00      0.00      0.00      1444
           5       0.00      0.00      0.00       588
           6       0.00      0.00      0.00       747
           7       0.00      0.00      0.00      1208
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00      1456
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00      3698
          13       0.00      0.00      0.00      1437
          14       0.00      0.00      0.00       174
          15       0.22      1.00      0.36      4342
          16       0.00      0.00      0.00       530
          17       0.00    

# 4. TF-IDF/Bag of words + Logistic Regression

In [65]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(random_state=random_state)
X_train_vec = vectorizer.fit_transform(
    X_train["title_lemma"] + " " + X_train["text_lemma"]
)
X_val_vec = vectorizer.transform(X_val["title_lemma"] + " " + X_val["text_lemma"])

lr_clf = LogisticRegression(max_iter=1000, random_state=random_state)
lr_clf.fit(X_train_vec, y_train)

y_val_pred = lr_clf.predict(X_val_vec)
print(classification_report(y_val, y_val_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.75      0.34      0.47        35
           2       0.00      0.00      0.00         2
           3       0.58      0.39      0.47       200
           4       0.81      0.81      0.81      1444
           5       0.84      0.83      0.83       588
           6       0.62      0.56      0.59       747
           7       0.77      0.71      0.74      1208
           8       0.80      0.22      0.35        18
           9       0.00      0.00      0.00         9
          10       0.87      0.87      0.87      1456
          11       0.00      0.00      0.00         3
          12       0.78      0.81      0.80      3698
          13       0.82      0.81      0.81      1437
          14       0.80      0.70      0.74       174
          15       0.76      0.82      0.79      4342
          16       0.63      0.51      0.56       530
          17       0.97    

In [66]:
# TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(random_state=random_state)
X_train_tfidf = tfidf_vectorizer.fit_transform(
    X_train["title_lemma"] + " " + X_train["text_lemma"]
)
X_val_tfidf = tfidf_vectorizer.transform(
    X_val["title_lemma"] + " " + X_val["text_lemma"]
)

lr_clf = LogisticRegression(max_iter=1000, random_state=random_state)
lr_clf.fit(X_train_tfidf, y_train)

y_val_pred = lr_clf.predict(X_val_tfidf)
print(classification_report(y_val, y_val_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00         2
           3       0.70      0.14      0.23       200
           4       0.82      0.81      0.81      1444
           5       0.86      0.77      0.81       588
           6       0.66      0.52      0.58       747
           7       0.80      0.72      0.76      1208
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         9
          10       0.86      0.89      0.88      1456
          11       0.00      0.00      0.00         3
          12       0.79      0.85      0.82      3698
          13       0.82      0.85      0.83      1437
          14       0.82      0.49      0.62       174
          15       0.76      0.85      0.80      4342
          16       0.73      0.33      0.46       530
          17       0.97    

# 5. Cross-validation

In [82]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin


def combine_features(X):
    return X["title_lemma"] + " " + X["text_lemma"]


def custom_tokenize(text):
    text = re.sub(r"[^а-яА-Я ]", "", text)
    return text.split()


# Создаем класс, который выбирает между разными векторизаторами
class VectorizerSelector(BaseEstimator, TransformerMixin):
    def __init__(self, vectorizer_type="tfidf", max_df=1.0, tokenizer=None, **kwargs):
        self.vectorizer_type = vectorizer_type
        self.max_df = max_df
        self.tokenizer = tokenizer
        self.kwargs = kwargs
        self.vectorizer = None

    def fit(self, X, y=None):
        if self.vectorizer_type == "count":
            self.vectorizer = CountVectorizer(
                max_df=self.max_df, tokenizer=self.tokenizer, **self.kwargs
            )
        else:
            self.vectorizer = TfidfVectorizer(
                max_df=self.max_df, tokenizer=self.tokenizer, **self.kwargs
            )
        self.vectorizer.fit(X)
        return self

    def transform(self, X):
        return self.vectorizer.transform(X)


pipeline = Pipeline(
    [
        ("vect", VectorizerSelector()),
        ("classifier", LogisticRegression(max_iter=1000, n_jobs=6)),
    ]
)

param_grid = {
    "vect__vectorizer_type": ["count", "tfidf"],
    "vect__max_df": [0.5, 0.7],
    "vect__tokenizer": [None, custom_tokenize],
}

random_search = RandomizedSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="f1_weighted",
    verbose=2,
    n_jobs=6,
    random_state=random_state,
)

X_train_combined = combine_features(X_train)

print("Начинаем поиск оптимальных гиперпараметров...")
random_search.fit(X_train_combined, y_train)

print("\nЛучшие параметры:")
print(random_search.best_params_)
print(f"\nЛучший f1-weighted score: {random_search.best_score_:.4f}")

best_model = random_search.best_estimator_

Начинаем поиск оптимальных гиперпараметров...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


Python(24160) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24161) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24162) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24163) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24164) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24165) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=count; total time= 1.2min
[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=count; total time= 1.2min
[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=count; total time= 1.2min
[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=count; total time= 1.3min
[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=count; total time= 1.3min


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END vect__max_df=0.5, vect__tokenizer=None, vect__vectorizer_type=tfidf; total time= 1.6min


...

[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x1373e72e0>, vect__vectorizer_type=count; total time= 1.1min


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x11d8872e0>, vect__vectorizer_type=count; total time= 1.1min


/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x12e2e32e0>, vect__vectorizer_type=count; total time= 1.1min
[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x10f9bf2e0>, vect__vectorizer_type=tfidf; total time= 1.2min
[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x100f73ec0>, vect__vectorizer_type=tfidf; total time= 1.0min
[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x12f13e7a0>, vect__vectorizer_type=tfidf; total time=  57.8s
[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x100cfbec0>, vect__vectorizer_type=tfidf; total time=  48.1s
[CV] END vect__max_df=0.7, vect__tokenizer=<function custom_tokenize at 0x11d89cc20>, vect__vectorizer_type=tfidf; total time=  47.4s

Лучшие параметры:
{'vect__vectorizer_type': 'tfidf', 'vect__tokenizer': None, 'vect__max_df': 0.5}

Лучший f1-weighted score: 0.7962


Лучшие параметры:
{'vect__vectorizer_type': 'tfidf', 'vect__tokenizer': None, 'vect__max_df': 0.5}

Лучший f1-weighted score: 0.7962

# 6. Оценка на тестовой выборке

In [83]:
X_test_combined = combine_features(X_test)

y_test_pred = best_model.predict(X_test_combined)

print("\nРезультаты на тестовой выборке:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Результаты на тестовой выборке:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       1.00      0.06      0.11        34
           2       0.00      0.00      0.00         2
           3       0.88      0.22      0.35       200
           4       0.81      0.82      0.82      1445
           5       0.86      0.74      0.79       588
           6       0.71      0.56      0.63       747
           7       0.78      0.72      0.75      1209
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         9
          10       0.85      0.88      0.87      1455
          11       0.00      0.00      0.00         3
          12       0.80      0.85      0.82      3697
          13       0.83      0.85      0.84      1438
          14       0.78      0.43      0.56       173
          15       0.77      0.85      0.80      4342
          16       0.71      0.34      0.46     